# Iceberg Lakehouse with Spark Thrift Server

This notebook demonstrates how to work with Apache Iceberg tables using Spark Thrift Server, MinIO for storage, and Nessie as the catalog.

We'll connect to the Spark Thrift Server instead of creating a local Spark session.

In [5]:
# Install required packages for Thrift server connection
!pip install pyhive thrift sasl

  Using cached sasl-0.3.1.tar.gz (44 kB)
  Preparing metadata (setup.py) ... Collecting sasl
  Using cached sasl-0.3.1.tar.gz (44 kB)
  Preparing metadata (setup.py) ... -done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [30 lines of output]
      /private/var/folders/04/9v689q8160l_tgg1fr9zmj1w0000gn/T/pip-install-vkxnkpaw/sasl_11d120e28f014d5c8beba05a70ea7069/setup.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        current_system = LooseVersion(platform.mac_ver()[0])
      /private/var/folders/04/9v689q8160l_tgg1fr9zmj1w0000gn/T/pip-install-vkxnkpaw/sasl_11d120e28f014d5c8beba05a70ea7069/setup.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        python_target = LooseVersion(
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-11.1-arm64-cpython-312/sa

In [9]:
from pyhive import hive
import pandas as pd
import time

# Connection configuration
THRIFT_HOST = 'spark-thrift'
THRIFT_PORT = 10000

def get_connection():
    """Get connection to Spark Thrift Server"""
    try:
        conn = hive.Connection(
            host=THRIFT_HOST,
            port=THRIFT_PORT,
            username='spark'
        )
        return conn
    except Exception as e:
        print(f"Failed to connect to Thrift server: {e}")
        return None

def execute_query(query, fetch=True):
    """Execute a query on Spark Thrift Server"""
    conn = get_connection()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        if fetch:
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(results, columns=columns)
            return df
        else:
            return "Query executed successfully"
    except Exception as e:
        print(f"Query failed: {e}")
        return None
    finally:
        conn.close()

print("Functions defined. Ready to connect to Spark Thrift Server.")

Functions defined. Ready to connect to Spark Thrift Server.


In [10]:
# Test connection to Thrift server
print("Testing connection to Spark Thrift Server...")
result = execute_query("SHOW DATABASES")
if result is not None:
    print("✅ Connection successful!")
    print("Available databases:")
    print(result)
else:
    print("❌ Connection failed")

failed to resolve sockaddr for spark-thrift:10000
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.12/site-packages/thrift/transport/TSocket.py", line 123, in open
    addrs = self._resolveAddr()
            ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.12/site-packages/thrift/transport/TSocket.py", line 37, in _resolveAddr
    return socket.getaddrinfo(self.host,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.12/socket.py", line 976, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 8] nodename nor servname provided, or not known


Testing connection to Spark Thrift Server...
Failed to connect to Thrift server: failed to resolve sockaddr for spark-thrift:10000
❌ Connection failed


In [ ]:
# Test Nessie catalog configuration
print("Testing Nessie catalog...")
result = execute_query("SHOW NAMESPACES IN nessie")
if result is not None:
    print("✅ Nessie catalog accessible!")
    print("Available namespaces:")
    print(result)
else:
    print("❌ Nessie catalog not accessible")

In [ ]:
# Create a namespace for our demo
print("Creating demo namespace...")
result = execute_query("CREATE NAMESPACE IF NOT EXISTS nessie.demo", fetch=False)
print(result)

# Verify namespace creation
result = execute_query("SHOW NAMESPACES IN nessie")
print("\nNamespaces in Nessie:")
print(result)

In [ ]:
# Create sample Iceberg table with sales data
create_table_query = """
CREATE TABLE IF NOT EXISTS nessie.demo.sales (
    id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    quantity INT,
    price DECIMAL(10,2),
    sale_date DATE,
    created_at TIMESTAMP
) USING ICEBERG
TBLPROPERTIES (
    'format-version'='2'
)
"""

print("Creating Iceberg table...")
result = execute_query(create_table_query, fetch=False)
print(result)

In [ ]:
# Insert sample data
insert_query = """
INSERT INTO nessie.demo.sales VALUES
    (1, 101, 'Laptop', 1, 999.99, '2024-01-15', current_timestamp()),
    (2, 102, 'Mouse', 2, 25.50, '2024-01-15', current_timestamp()),
    (3, 103, 'Keyboard', 1, 75.00, '2024-01-16', current_timestamp()),
    (4, 101, 'Monitor', 1, 299.99, '2024-01-16', current_timestamp()),
    (5, 104, 'Headphones', 1, 149.99, '2024-01-17', current_timestamp())
"""

print("Inserting sample data...")
result = execute_query(insert_query, fetch=False)
print(result)

In [ ]:
# Query the data
print("Querying sales data...")
result = execute_query("SELECT * FROM nessie.demo.sales ORDER BY id")
if result is not None:
    print("Sales data:")
    print(result)
else:
    print("Failed to query data")

In [ ]:
# Demonstrate time travel - get table history
print("Getting table history (snapshots)...")
result = execute_query("SELECT * FROM nessie.demo.sales.history")
if result is not None:
    print("Table history:")
    print(result)
else:
    print("Failed to get table history")

In [ ]:
# Add more data to demonstrate time travel
print("Adding more data...")
insert_query2 = """
INSERT INTO nessie.demo.sales VALUES
    (6, 105, 'Webcam', 1, 89.99, '2024-01-18', current_timestamp()),
    (7, 106, 'Speaker', 2, 45.00, '2024-01-18', current_timestamp())
"""

result = execute_query(insert_query2, fetch=False)
print(result)

# Show current data
print("\nCurrent data:")
result = execute_query("SELECT COUNT(*) as total_records FROM nessie.demo.sales")
print(result)

In [ ]:
# Demonstrate aggregation query
print("Sales summary by customer:")
summary_query = """
SELECT 
    customer_id,
    COUNT(*) as num_orders,
    SUM(quantity) as total_items,
    SUM(price * quantity) as total_amount
FROM nessie.demo.sales 
GROUP BY customer_id 
ORDER BY total_amount DESC
"""

result = execute_query(summary_query)
if result is not None:
    print(result)
else:
    print("Failed to execute summary query")

In [ ]:
# Show table metadata
print("Table metadata:")
result = execute_query("DESCRIBE EXTENDED nessie.demo.sales")
if result is not None:
    print(result)
else:
    print("Failed to get table metadata")

In [ ]:
# Test schema evolution - add a new column
print("Adding new column to demonstrate schema evolution...")
alter_query = "ALTER TABLE nessie.demo.sales ADD COLUMN discount_percent DECIMAL(5,2)"

result = execute_query(alter_query, fetch=False)
print(result)

# Verify schema change
print("\nUpdated schema:")
result = execute_query("DESCRIBE nessie.demo.sales")
if result is not None:
    print(result)
else:
    print("Failed to describe table")

In [ ]:
# Update some records with discount
print("Updating records with discount information...")
update_query = """
UPDATE nessie.demo.sales 
SET discount_percent = 10.0 
WHERE customer_id IN (101, 102)
"""

result = execute_query(update_query, fetch=False)
print(result)

# Show updated data
print("\nUpdated data:")
result = execute_query("SELECT * FROM nessie.demo.sales WHERE discount_percent IS NOT NULL")
if result is not None:
    print(result)
else:
    print("Failed to query updated data")

## Summary

This notebook demonstrated:

1. **Connection to Spark Thrift Server** - Using PyHive to connect to the Thrift server
2. **Iceberg Table Creation** - Creating tables with format version 2
3. **Data Operations** - INSERT, UPDATE, and SELECT operations
4. **Schema Evolution** - Adding new columns to existing tables
5. **Time Travel** - Accessing table history and snapshots
6. **Aggregations** - Complex queries with GROUP BY and aggregations

The setup successfully demonstrates a modern lakehouse architecture with:
- **Apache Iceberg** for table format with ACID transactions
- **Nessie** as the REST catalog for metadata management
- **MinIO** as S3-compatible object storage
- **Spark Thrift Server** for SQL query execution
- **Jupyter** for interactive data exploration